In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,56717
2,Huelva,Confirmados PDIA 14 días,6495
3,Huelva,Tasa PDIA 14 días,"1265,6624510396166"
4,Huelva,Confirmados PDIA 7 días,3579
5,Huelva,Tasa PDIA 7 dias,"697,4297016583198"
6,Huelva,Total Confirmados,56926
7,Huelva,Curados,47756
8,Huelva,Fallecidos,427


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  56717.0


/tmp/ipykernel_74546/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  16167.0


/tmp/ipykernel_74546/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_74546/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_74546/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_74546/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 131 personas en los últimos 7 días 

Un positivo PCR cada 68 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,56717.0,6495.0,3579.0,513170.0,697.429702,1265.662451,2605.0
Huelva-Costa,33420.0,3824.0,2008.0,289548.0,693.494688,1320.679127,1563.0
Huelva (capital),16167.0,2103.0,1097.0,143837.0,762.668854,1462.071651,839.0
Condado-Campiña,17123.0,1559.0,937.0,156231.0,599.752930,997.881342,655.0
Sierra de Huelva-Andévalo Central,5733.0,1102.0,630.0,67391.0,934.842932,1635.233191,382.0
Ayamonte,2540.0,336.0,156.0,21104.0,739.196361,1592.115239,128.0
Aljaraque,2159.0,330.0,156.0,21474.0,726.459905,1536.742107,125.0
Valverde del Camino,1282.0,336.0,186.0,12750.0,1458.823529,2635.294118,116.0
Isla Cristina,3445.0,196.0,133.0,21393.0,621.698686,916.187538,113.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,75.0,49.0,16.0,614.0,2605.863192,7980.456026,10.0
Fuenteheridos,45.0,24.0,7.0,709.0,987.306065,3385.049365,3.0
Puerto Moral,25.0,9.0,4.0,278.0,1438.848921,3237.410072,2.0
Galaroza,102.0,39.0,32.0,1382.0,2315.484805,2821.997106,17.0
Valverde del Camino,1282.0,336.0,186.0,12750.0,1458.823529,2635.294118,116.0
Escacena del Campo,294.0,49.0,31.0,2287.0,1355.487538,2142.544819,9.0
Manzanilla,206.0,45.0,27.0,2118.0,1274.787535,2124.645892,18.0
Campofrío,77.0,15.0,14.0,713.0,1963.534362,2103.786816,10.0
Aracena,1023.0,172.0,73.0,8255.0,884.312538,2083.585706,37.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Santa Bárbara de Casa,53.0,5.0,1.0,1043.0,95.877277,479.386385,1.0,0.200000
Fuenteheridos,45.0,24.0,7.0,709.0,987.306065,3385.049365,3.0,0.291667
San Silvestre de Guzmán,75.0,49.0,16.0,614.0,2605.863192,7980.456026,10.0,0.326531
Aracena,1023.0,172.0,73.0,8255.0,884.312538,2083.585706,37.0,0.424419
Sanlúcar de Guadiana,42.0,7.0,3.0,403.0,744.416873,1736.972705,2.0,0.428571
Higuera de la Sierra,91.0,14.0,6.0,1291.0,464.756003,1084.430674,4.0,0.428571
Cortegana,399.0,60.0,26.0,4602.0,564.971751,1303.780965,14.0,0.433333
Cumbres Mayores,211.0,32.0,14.0,1749.0,800.457404,1829.616924,8.0,0.437500
Puerto Moral,25.0,9.0,4.0,278.0,1438.848921,3237.410072,2.0,0.444444
